In [19]:
#데이터 핸들링 라이브러리
import pandas as pd
import numpy as np
#데이터 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.family':'AppleGothic'})
mpl.rc('axes', unicode_minus=False)
%config InlineBackend.figure_format = 'retina'


import sys
sys.path.append('/Users/wooseongkyun/코드_아카이브/프로젝트2_머신러닝과EDA/binomial_classification/preprocessing_data.py')

In [20]:
#기후 데이터를 가져오기
df= pd.read_csv('./기상청_중기예보_최근5년데이터.csv',encoding='euc-kr')
df.drop(['강수확률(%)','발표시각','신뢰도'],axis=1,inplace=True)

/var/folders/m9/0vsmjxhd7q30k5mzw0bk58hh0000gn/T/ipykernel_66403/1886206877.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('./기상청_중기예보_최근5년데이터.csv',encoding='euc-kr')


In [21]:
AM_PM_lst=[]
day_lst=[]

date_lst=df['예보시각']
for date in date_lst:
    splited_date= date.split()
    day_lst.append(splited_date[0])
    AM_PM_lst.append(splited_date[-1])

df['예보일']=day_lst
df['오전_오후']=AM_PM_lst

#오전오후를 0과 1값으로 표현하기
is_PM =df['오전_오후']=='오후'
df['오전0_오후1']=is_PM
df['오전0_오후1']= df['오전0_오후1'].astype(int)

#중복된 데이터정보를 나타내는 열들 삭제하기
#계산을 간략화하기 위해 지역에 대한 가중치를 모두 같게함. 그로써 지역에 대한 컬럼을 삭제
df.drop(['예보시각','오전_오후','지역','오전0_오후1'],axis=1,inplace=True)
df.sort_values(by='예보일',ascending=True)

,예보,예보일
0,맑음,2017-01-04
52,맑음,2017-01-04
53,맑음,2017-01-04
65,맑음,2017-01-04
66,맑음,2017-01-04
...,...,...
474642,맑음,2022-01-10
474720,구름많음,2022-01-10
474707,구름많음,2022-01-10
474746,맑음,2022-01-10


In [26]:
#기상 관측 분류를 원-핫 인코딩하기
features=['예보']
encoded_df= pd.get_dummies(data=df,columns=features,prefix=None)
encoded_df

#각 지역별 관측된 기상현상값들을 동일한 가중치(1)로 두고 더하여 전국 날씨값을 계산하기
encoded_df2= encoded_df.copy()
encoded_df2['맑음']= encoded_df2['예보_맑음']+encoded_df2['예보_구름조금']+encoded_df2['예보_구름많음']
encoded_df2['비']=encoded_df2['예보_구름많고 비']+encoded_df2['예보_구름많고 소나기']+\
                  encoded_df2['예보_흐리고 비']+encoded_df2['예보_흐리고 소나기']
encoded_df2['흐림']=encoded_df2['예보_흐림']
encoded_df2['눈']=encoded_df2['예보_구름많고 눈']+encoded_df2['예보_구름많고 눈/비']+\
                  encoded_df2['예보_구름많고 비/눈']+encoded_df2['예보_흐리고 눈']+\
                  encoded_df2['예보_흐리고 눈/비']+encoded_df2['예보_흐리고 비/눈']

encoded_df2['비/눈']=encoded_df2['눈']+encoded_df2['비']
encoded_df2= encoded_df2[['예보일','맑음','흐림','비/눈']]
encoded_df2.groupby(by='예보일').value_counts()

예보일         맑음  흐림  비/눈
2017-01-04  1   0   0       40
2017-01-05  1   0   0       71
            0   0   1        9
2017-01-06  1   0   0      120
2017-01-07  1   0   0      143
                          ... 
2022-01-07  0   1   0        1
2022-01-08  1   0   0       60
2022-01-09  1   0   0       40
2022-01-10  1   0   0       19
            0   1   0        1
Length: 3587, dtype: int64

In [ ]:
#각 지역별 관측된 기상현상값들을 동일한 가중치(1)로 두고 더하여 전국 날씨값을 계산하기
evaluated_df= encoded_df.groupby(by='예보일').sum()
evaluated_df['맑음']=  evaluated_df['예보_맑음']+evaluated_df['예보_구름조금']+evaluated_df['예보_구름많음']
evaluated_df['비']=evaluated_df['예보_구름많고 비']+evaluated_df['예보_구름많고 소나기']+\
                  evaluated_df['예보_흐리고 비']+evaluated_df['예보_흐리고 소나기']
evaluated_df['흐림']=evaluated_df['예보_흐림']
evaluated_df['눈']=evaluated_df['예보_구름많고 눈']+evaluated_df['예보_구름많고 눈/비']+\
                  evaluated_df['예보_구름많고 비/눈']+evaluated_df['예보_흐리고 눈']+\
                  evaluated_df['예보_흐리고 눈/비']+evaluated_df['예보_흐리고 비/눈']


evaluated_df= evaluated_df[['맑음','비','흐림','눈']]
evaluated_df

In [ ]:
weather_df=evaluated_df.idxmax(axis=1)
weather_lst = list(weather_df.values)

In [ ]:
weather_df=pd.DataFrame(weather_df)
weather_df['기상']=weather_lst
weather_df.drop(0,axis=1,inplace=True)

In [ ]:
year_lst=[]
date_lst=list(weather_df.index)
for date in date_lst:
    splited_date= str(date)[0:4]
    year_lst.append(splited_date)

year_lst
weather_df['년도']= year_lst

weather_category=weather_df.groupby('년도').value_counts()
weather_category.loc['2020']
weather_category

In [ ]:
weather_category.to_csv("기상청_전처리된데이터.csv", mode='w')

In [ ]:
list(weather_df.index)